<a href="https://colab.research.google.com/github/veroorli/ProjetProg/blob/master/Copie_de_TP9_10_2022_partition_fenetre_jointure_ETUDIANT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BDLE 2022

date du document  :  16/12/2022

# TP 9 et 10 : traitement sur des partitions : tri, regroupement, fenetre, jointure
Version ETUDIANT



# Indications, méthode

L'objectif est de comprendre la notion de traitement sur des données partitionnées.
Savoir *décomposer* un traitement complexe (une requête SQL) en une suite d'étapes. 
Savoir séparer les étapes de **traitement dans une partition** des étapes de **repartitionnement des données**. Dans une étape de traitement, on peut si nécesaire calculer un atribut qui servira à un repartitionnement ultérieur.



Indications:

*   Les données sont gérées par spark et sont supposées être très volumineuses. Elle ne doivent jamais être "remontées" entièrement dans l'application. Ne jamais invoquer un collet() sur la totalité des données. 
*   On peut remonter dans l'application le résultat d'une requête si on sait que sa taille est petite.
*   On peut faire "descendre" vers spark des données auxilliaires provenant de l'application (supposées de petite taille). Ces données auxilliaires pourront ensuite être lues lors d'un prochain traitement dans une partition.


Implémentation :

Savoir définir une fonction UDF qui implémente le traitement dans une partition et qui est invoquée avec mapPartition ou mapPartitionWithIndex.

La fonction repartition permet de repartitionner les données

La fonction broadcast permet de diffuser des données auxilliaires




##  Indications pour la Q1 (tri)

Exemple correct 
*   Trier localement les données par intervalle
*   Repartitionner les données en fonction du numéro d'intervalle
*   Dans chaque intervalle, fusionner les données.

Exemple **incorrect**
*   Remonter les données dans l'application avec un collect
*   Trier les données dans l'application
*   Re-descendre les données dans spark avec un createDataFrame

##  Indications pour la Q5 (mots fréquents)

Exemple générant **trop de transferts**  :
*   Découper le titre en liste de mots pour obtenir des couples (film,mot)
*   repartitionner les données en fonction d'un mot du titre : attention, les données repartionnées sont trop volumineuses.
*   Compter la frequence des mots dans chaque partition

Exemple correct  :
*   Découper le titre en liste de mots puis compter la fréquence des mots dans chaque partition. On obtient des couples (mot, fréquence)
*   repartitionner les couples (mot, fréquence) en fonction d'un mot
*   Additionner les fréquences pour chaque mot et les trier pour ne garder que les 5 plus fréquents. 
*   transférer vers l'application les 5 mots plus fréquents de chaque partition puis fusionner les listes pour connaitre les 5 mots globalement les plus fréquents.



##  Indications pour la Q6: jointure
 * Attribuer à chaque film et chaque note un numéro de partition en fonction de nF. Par exemple le numéro de partition peut être : nF modulo le nombre de partition.
 * Repartitionner les données en fonction du numéro de partition

 * Former des paires de partitions de film et de note telles qu'elles ont le même numéro de partition. Puis calculer la jointure.


Remarque : Un solution correcte mais pas la plus rapide (car elle fait une jointure par boucles imbriquée qui est souvent mois rapide qu'une jointure par hachage ou par tri fusion) est de calculer tous les couples (film,note) qui ont le meme numéro de partition (au lieu de former des paires de partition film/note). Il suffit ensuite calculer la jointure en filtrant les couples qui satisfont la condition de jointure.

Ne pas faire de jointure sur le nF (qui est la solution déjà prête avec laquelle il faut vous comparer...)

# Préparation

Pour accéder directement aux fichiers stockées sur votre google drive. Renseigner le code d'authentification lorsqu'il est demandé

Ajuster le nom de votre dossier : MyDrive/essai

In [ ]:
# import os
# from google.colab import drive
# drive.mount("/content/drive")

# drive_dir = "/content/drive/MyDrive/essai"
# os.makedirs(drive_dir, exist_ok=True)
# os.listdir(drive_dir)

Installer pyspark et findspark :


In [ ]:
!pip install -q pyspark
!pip install -q findspark
print("installé")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 8.5 MB/s eta 0:00:00
installé


Démarrer la session spark

In [ ]:
import os
# !find /usr/local/lib/ -name "pyspark" 
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.8/dist-packages/pyspark"
os.environ["JAVA_HOME"] = "/usr"

In [ ]:
# Principaux import
import findspark
from pyspark.sql import SparkSession 
from pyspark import SparkConf  

# pour les dataframe et udf
from pyspark.sql import *  
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *

# pour le chronomètre
import time

# initialise les variables d'environnement pour spark
findspark.init()

# Démarrage session spark 
# --------------------------
def demarrer_spark():
  local = "local[*]"
  appName = "TP"
  configLocale = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "6G").\
  set("spark.driver.memory","3G").\
  set("spark.sql.catalogImplementation","in-memory")
  
  spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")
  
  spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")

  # On ajuste l'environnement d'exécution des requêtes à la taille du cluster (4 coeurs)
  spark.conf.set("spark.sql.shuffle.partitions","4")    
  print("session démarrée, son id est ", sc.applicationId)
  return spark
spark = demarrer_spark()

session démarrée, son id est  local-1673868183148


Redéfinir la fonction **display** pour afficher le resutltat des requêtes dans un tableau

In [ ]:
import pandas as pd
from google.colab import data_table

# alternatives to Databricks display function.

def display(df, n=100):
  return data_table.DataTable(df.limit(n).toPandas(), include_index=False, num_rows_per_page=10)

def display2(df, n=20):
  pd.set_option('max_columns', None)
  pd.set_option('max_colwidth', None)
  return df.limit(n).toPandas().head(n)

Définir le tag **%%sql** pour pouvoir écrire plus simplement des requêtes en SQL dans une cellule

In [ ]:
from IPython.core.magic import (register_line_magic, register_cell_magic, register_line_cell_magic)

def removeComments(query):
  result = ""
  for line in query.split('\n'):
    if not(line.strip().startswith("--")):
      result += line + "\n"
  return result

@register_line_cell_magic
def sql(line, cell=None):
    "To run a sql query. Use:  %%sql"
    val = cell if cell is not None else line
    tabRequetes = removeComments(val).split(";")
    derniere = None
    est_requete = False
    for r in tabRequetes:
        r = r.strip()
        if len(r) > 2:
          derniere = spark.sql(r)
          est_requete = ( r.lower().startswith('select') or r.lower().startswith('with') )
    if(est_requete):
      return display(derniere)
    else:
      return print('ok')

Utiliaires : Chronomètres

In [ ]:
#------------------------------
# Chronometre : chronoPersist2
#------------------------------
import time

# Ce chronometre garantit que chaque tuple du dataframe est lu entièrement.
# En effet il est nécessaire de lire le détail de chaque tuple avant de les 'copier' en mémoire.
def chronoPersist(df):
    df.unpersist()
    t1 = time.perf_counter()
    count = df.persist().count()
    t2 = time.perf_counter()
    df.unpersist()
    print('durée: {:.1f} s'.format(t2 - t1), 'pour lire', count , 'elements')

def chronoPersist2(df):
  dest = df.selectExpr("*", "1")
  t1 = time.perf_counter()
  count = dest.persist().count()
  t2 = time.perf_counter()
  dest.unpersist()
  print('durée: {:.1f} s'.format(t2 - t1), 'pour lire', count , 'elements')
        
def chronoCount(df):
  t1 = time.perf_counter()
  count = df.count()
  t2 = time.perf_counter()
  print('durée: {:.1f} s'.format(t2 - t1), 'pour dénombrer', count , 'elements')
    
print("fonctions définies")

fonctions définies


# Accès aux données

In [ ]:
import os
local_dir = "/local/data"
os.makedirs(local_dir, exist_ok=True)
os.listdir(local_dir)

[]

URL pour l'accès aux datasets

In [ ]:
# ---------------------------------------------------------------------------
# en cas de problème avec le téléchargement des datasets, aller directement sur l'URL ci-dessous
PUBLIC_DATASET_URL = "https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4" 
PUBLIC_DATASET=PUBLIC_DATASET_URL + "/download?path="

print("URL pour les datasets ", PUBLIC_DATASET_URL)

URL pour les datasets  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4


In [ ]:
import os
from urllib import request
import zipfile

# download dataset if not already donwloaded
def download_file(web_dir, local_dir, file):
  local_file = local_dir + "/" + file
  web_file = web_dir + "/" + file
  if(os.path.isfile(local_file)):
    print(file, "is already stored")
  else:
    print("downloading from URL: ", web_file , "save in : " + local_file)
    request.urlretrieve(web_file , local_file)

def unzip_file(local_dir, file):
  with zipfile.ZipFile(local_dir + "/" + file, 'r') as zip_ref:
    zip_ref.extractall(local_dir)
  # os.remove(local_dir + "/" + file)


web_dir = PUBLIC_DATASET + "/movielens"

download_file(web_dir, local_dir, "notes1M.zip")
unzip_file(local_dir, "notes1M.zip")

download_file(web_dir, local_dir, "ratings3M.zip")
unzip_file(local_dir, "ratings3M.zip")

download_file(web_dir, local_dir, "films.json")

web_dir = PUBLIC_DATASET + "/movielens/ml-latest"

download_file(web_dir, local_dir, "movies.csv")


# Liste des fichiers
os.listdir(local_dir)

downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=/movielens/notes1M.zip save in : /local/data/notes1M.zip
downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=/movielens/ratings3M.zip save in : /local/data/ratings3M.zip
downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=/movielens/films.json save in : /local/data/films.json
downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=/movielens/ml-latest/movies.csv save in : /local/data/movies.csv


['notes1M.zip',
 'notes1M.json',
 'ratings3M.zip',
 'ratings3M.csv',
 'films.json',
 'movies.csv']

### Dataframe les Films

In [ ]:
# schema_film = "nf long, titre String, g Array<String>"
schema_film = StructType([StructField('nF',LongType()),
                          StructField('titre',StringType()),
                          StructField('g',ArrayType(StringType()))])

films = spark.read.json(local_dir + "/" + "films.json", schema = schema_film).selectExpr("nF", "titre", "g as genres")
# print('schema:', films.schema)

# extrait
film_extrait = films.where("nF <100").repartition(3).persist()

film_extrait.printSchema()
display(film_extrait)

root
 |-- nF: long (nullable = true)
 |-- titre: string (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)



,nF,titre,genres
0,94,Beautiful Girls (1996),"[Comedy, Drama, Romance]"
1,45,To Die For (1995),"[Comedy, Drama, Thriller]"
2,58,"Postman, The (Postino, Il) (1994)","[Comedy, Drama, Romance]"
3,54,"Big Green, The (1995)","[Children, Comedy]"
4,9,Sudden Death (1995),[Action]
...,...,...,...
87,38,It Takes Two (1995),"[Children, Comedy]"
88,15,Cutthroat Island (1995),"[Action, Adventure, Romance]"
89,85,Angels and Insects (1995),"[Drama, Romance]"
90,64,Two if by Sea (1996),"[Comedy, Romance]"


### Dataframe Les Notes

In [ ]:
notes_schema = StructType([StructField('nF',LongType()),
                           StructField('nU',LongType()),
                           StructField('note',DoubleType()),
                           StructField('annee',LongType())])

notes = spark.read.json(local_dir + "/" + "notes1M.json", schema = notes_schema).selectExpr("nF", "nU", "note", "annee")

#extrait
notes_extrait = notes.where("nU < 1000").join(film_extrait, "nF").select(notes["nF"], "nU", "note", "annee").repartition(3).persist()
print(notes_extrait.count())
display(notes_extrait)

185


,nF,nU,note,annee
0,19,76,1.5,2011
1,62,691,5.0,1996
2,25,832,5.0,1998
3,5,296,3.5,2005
4,48,781,3.0,1996
...,...,...,...,...
95,69,798,3.0,2000
96,95,430,3.0,1996
97,10,984,4.0,1996
98,36,261,2.0,1997


# Exercice 1 : Traitement sur des partitions

#### Fonction showPartitionSize

La fonction *showPartitionSize*  affiche le nombre d'éléments dans chaque partition

In [ ]:
# La fonction part_size prend en paramètres un numéro de partition (partID) et un itérateur sur une partition.
# Elle retourne un itérateur sur une partition qui contient un seul tuple (num_partition, nbtuples).

def partSize(partID, iterateur):
  size=0
  suivant = next(iterateur, None)
  while suivant is not None :
    size += 1
    suivant = next(iterateur, None)
  yield (partID, size)


def showPartitionSize(df):
  if df.isEmpty():
    print("empty dataframe")
  else:
    #invoquer la fonction partSize sur chaque partition 
    t = df.selectExpr("1").rdd.mapPartitionsWithIndex(partSize).toDF(['partID', 'size'])

    # Rmq : selectExpr("1") sert à simplifier la partition pour ne garder qu'une seule colonne contentant des "1",
    # ce qui est suffisant pour compter le nombre de tuples dans la partition.

    #affichage
    t.show()

print('showPartitionSize définie')

showPartitionSize définie


#### Fonction showPartitions
 
La fonction *showPartitions*  affiche les _n_ premiers éléments de chaque partition


In [ ]:
def showPartitions(df, N=5 , display_per_partition=True):
  if df.isEmpty():
     print("empty dataframe")
  else:   
    nb_partitions = df.rdd.getNumPartitions()

    # fonction topN est invoquée sur une partition.
    # topN retourne un itérateur sur une partition qui contient les N premiers éléments de cette partition
    def topN(partID, iterateur):
      c=0
      suivant = next(iterateur, None)
      while suivant is not None and c < N :
        c+=1
        tuple_avec_numero_partition = (partID, *suivant)
        suivant = next(iterateur, None)
        yield tuple_avec_numero_partition
    #-- fin de topN ---

    partid_tmp = "_p_"
    nom_attributs = [partid_tmp] + df.schema.fieldNames()
    premiers_tuples = df.rdd.mapPartitionsWithIndex(topN).toDF(nom_attributs)
    premiers_tuples.persist()

    if(display_per_partition):
      # afficher séparément le contenu du début de chaque partition
      for partID in range(nb_partitions):
        print("partition", partID)
        # afficher la partition partID
        premiers_tuples.where(col(partid_tmp)==partID).drop(partid_tmp).show(N, False)
    else:
      return display(premiers_tuples)
   
print('showPartitions définie')

showPartitions définie


Afficher la taille de chaque partition

In [ ]:
showPartitionSize(film_extrait)

+------+----+
|partID|size|
+------+----+
|     0|  31|
|     1|  31|
|     2|  30|
+------+----+



Afficher les premiers éléments de chaque partition

In [ ]:
showPartitions(film_extrait,5)

partition 0
+---+---------------------------------+-------------------------+
|nF |titre                            |genres                   |
+---+---------------------------------+-------------------------+
|94 |Beautiful Girls (1996)           |[Comedy, Drama, Romance] |
|45 |To Die For (1995)                |[Comedy, Drama, Thriller]|
|58 |Postman, The (Postino, Il) (1994)|[Comedy, Drama, Romance] |
|54 |Big Green, The (1995)            |[Children, Comedy]       |
|9  |Sudden Death (1995)              |[Action]                 |
+---+---------------------------------+-------------------------+

partition 1
+---+--------------------------------------------+-----------------+
|nF |titre                                       |genres           |
+---+--------------------------------------------+-----------------+
|14 |Nixon (1995)                                |[Drama]          |
|71 |Fair Game (1995)                            |[Action]         |
|49 |When Night Is Falling (1995)   

## Question 1 : Tri des films par titre

En vous inspirant de la fonction *topN*  définir une fonction qui tri les données de *film_extrait* par titre. Le résultat a le meme schéma que film.

In [ ]:

def tri_par_titre(iterateur):
  suivant = next(iterateur, None)
  groupes = [[] for x in range(3)]
  while suivant is not None:
    lettre1 = suivant.titre[0]
    if lettre1 < 'H':
      p=0
    elif lettre1 < 'P':
      p=1
    else:
      p=2
    groupes[p].append((p, *suivant))
    suivant = next(iterateur, None)
  for groupe in groupes:
    for film in sorted(groupe, key=lambda t: t[2]):
      yield film

# tester la fonction
# it = iter(film_extrait.limit(3).collect())
# for x in tri_par_titre(it):
#   print(x)

names = ["p"] + film_extrait.schema.fieldNames()
film1 = film_extrait.rdd.mapPartitions(tri_par_titre).toDF(names)
showPartitions(film1, 30)

partition 0
+---+---+----------------------------------------------------+-------------------------------------------------+
|p  |nF |titre                                               |genres                                           |
+---+---+----------------------------------------------------+-------------------------------------------------+
|0  |13 |Balto (1995)                                        |[Adventure, Animation, Children]                 |
|0  |94 |Beautiful Girls (1996)                              |[Comedy, Drama, Romance]                         |
|0  |54 |Big Green, The (1995)                               |[Children, Comedy]                               |
|0  |88 |Black Sheep (1996)                                  |[Comedy]                                         |
|0  |35 |Carrington (1995)                                   |[Drama, Romance]                                 |
|0  |39 |Clueless (1995)                                     |[Comedy, Romance]     

repartitionner les données par **intervalle** en fonction de l'attribut p

In [ ]:
film2 = film1.repartitionByRange(3, col('p'))
showPartitions(film2,20)

partition 0
+---+---+--------------------------------------------+-----------------------------------------+
|p  |nF |titre                                       |genres                                   |
+---+---+--------------------------------------------+-----------------------------------------+
|0  |13 |Balto (1995)                                |[Adventure, Animation, Children]         |
|0  |94 |Beautiful Girls (1996)                      |[Comedy, Drama, Romance]                 |
|0  |54 |Big Green, The (1995)                       |[Children, Comedy]                       |
|0  |88 |Black Sheep (1996)                          |[Comedy]                                 |
|0  |35 |Carrington (1995)                           |[Drama, Romance]                         |
|0  |39 |Clueless (1995)                             |[Comedy, Romance]                        |
|0  |22 |Copycat (1995)                              |[Crime, Drama, Horror, Mystery, Thriller]|
|0  |36 |Dead Man 

In [ ]:
film_tri_total = film2.sortWithinPartitions('titre')
showPartitions(film_tri_total,15)

partition 0
+---+---+-------------------------------------+--------------------------------+
|p  |nF |titre                                |genres                          |
+---+---+-------------------------------------+--------------------------------+
|0  |19 |Ace Ventura: When Nature Calls (1995)|[Comedy]                        |
|0  |37 |Across the Sea of Time (1995)        |[Documentary, IMAX]             |
|0  |11 |American President, The (1995)       |[Comedy, Drama, Romance]        |
|0  |85 |Angels and Insects (1995)            |[Drama, Romance]                |
|0  |82 |Antonia's Line (Antonia) (1995)      |[Comedy, Drama]                 |
|0  |23 |Assassins (1995)                     |[Action, Crime, Thriller]       |
|0  |34 |Babe (1995)                          |[Children, Drama]               |
|0  |13 |Balto (1995)                         |[Adventure, Animation, Children]|
|0  |94 |Beautiful Girls (1996)               |[Comedy, Drama, Romance]        |
|0  |74 |Bed of 

## Question 2 : Regrouper les films par genre et les compter

En vous inspirant de la fonction *topN*  définir une fonction qui compte le nombre de films par genre. Le schéma du résultat est (genre, n)

In [ ]:
# def count_par_genre

## Question 3 : Numérotation

En vous inspirant de la fonction *topN*  définir une fonction qui attribue un numéro de 1 à n à chaque tuple de notes_extrait. Le résultat a les attributs de notes + un attribut num.

In [ ]:
# def numeroter

## Question 4 : Regrouper et trier

Définir une fonction qui affiche le nombre de notes par année dans l'ordre croissant des années

In [ ]:
# def regrouper_trier

## Question 5 : top fréquence

Définir une fonction qui affiche les 5 mots les plus fréquents dans les titres

In [ ]:
# def numeroter

## Question 6 : Jointure

Définir les fonctions pour calculer la jointure entre les extraits de films et de notes

In [ ]:
# def jointure_film_notes